<a href="https://colab.research.google.com/github/cesaroliveiragoes/LLM_Gemini_Alura/blob/main/Imers%C3%A3o_IA_Aura_Google_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
! pip install -q -U google-generativeai

In [41]:
# Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [42]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [43]:
# Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo complet:\n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA Generativa")

#Fazer o embedding do texto acima
embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type = "RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.051075656, -0.042413216, -0.019050116, 0.018651702, 0.06776935, -0.0076618, -0.025336018, -0.015564104, 0.0630246, 0.05966318, 0.011905264, 0.010113206, -0.04171998, -0.021177981, 0.005697867, -0.027380612, 0.021663442, -0.013841382, -0.03792506, 0.0016360482, 0.004748626, 0.012457731, -0.03385269, -0.06598702, -0.016129388, 0.02025235, 0.018276267, -0.042369887, -0.033043545, 0.020656472, -0.051115233, 0.049630936, -0.035193574, 0.009171094, -0.035946045, -0.041215666, -0.024667785, -0.0413526, -0.002867965, -0.004518409, 0.007448215, -0.08090475, -0.008665931, 0.027408067, -0.005392241, -0.023884539, 0.049526665, 0.042505905, 0.017980102, -0.05962825, 0.02730381, 0.021661555, 0.06510442, -0.0378072, -0.006709448, -0.0027959333, 0.018138722, -0.030888828, 0.02504816, 6.0484475e-05, -0.0022650082, 0.019256381, -0.02392349, 0.054552518, 0.015284966, -0.057963155, -0.03517785, -0.0031057484, 0.008491305, 0.050183427, 0.010411418, 0.015229794, 0.059401542, -0.059524935, -

In [44]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [45]:
df = pd.DataFrame(documents)
df

,title,content
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [46]:
model = "models/embedding-001"

In [47]:
def embed_fn(title, text):
  return genai.embed_content(model="models/embedding-001",
                             content=sample_text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [48]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["title"], row["content"]), axis=1) #Função aplicada para cada linha por coluna
df

,title,content,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[0.031596713, -0.039041594, -0.030192373, 0.03..."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.04943855, -0.061193287, -0.025331846, -0.00..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[0.03375102, -0.038284782, -0.019839277, 0.012..."


In [49]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=sample_text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta) # Calcular as distâncias

  indice = np.argmax(produtos_escalares) # o indice do token retorna o mais próximo
  return df.iloc[indice]["content"]

In [50]:
consulta = "How can I shift gears in a Google's car?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.


In [51]:
generation_config = {
    "temperature": 0,
    "candidate_count": 1,
}

In [52]:
# Juntar os resultados de Embedding e o Próprio Gemini para ser "criativo" nma resposta, não apenas uma cópia do data frame
prompt = f"Reescreva esse texto de forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)

response = model_2.generate_content(prompt)
print(response.text)

**Guia de marchas do seu Googlecar:**

* **Estacionado:** Quando você estaciona, é como se o carro estivesse dormindo. As rodas ficam travadas e ele não se mexe.
* **Ré:** Para dar aquela rézinha, é só colocar o câmbio nessa posição.
* **Neutro:** Quando você para no sinal ou no trânsito, é como se o carro estivesse de bobeira. Ele não está engatado e só vai andar se você pisar no acelerador.
* **Dirigir:** Essa é a marcha para você sair por aí.
* **Baixa:** Se você estiver dirigindo na neve ou em lugares escorregadios, essa marcha é a sua amiga.


In [53]:
# Embed é uma forma de representar a palavra por tokens, e criar uma representação vetorial
# trazer o contexto semântico das palavras
# Essa técnica faz com que entenda-se o contexto da palavra inserida
# Ele analisa a distância que temos de um token para o outro